In [1]:
import os
import pickle
from prettytable import PrettyTable
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import autocast
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

In [2]:
def sava_data(filename, data):
    """
    save given Python object data using pickle
    @param:
    - filename: filename, type:str
    - data: Python object to save, type:any
    """
    print("Begin to save data：", filename)
    f = open(filename, 'wb')
    pickle.dump(data, f)
    f.close()

def load_data(filename):
    """
    load Python object data from pickle file
    @param:
    - filename: filename, type:str
    """
    print("Begin to load data：", filename)
    f = open(filename, 'rb')
    data = pickle.load(f)
    f.close()
    return data

In [3]:
def get_accuracy(labels, prediction):
    """
    get accuracy score
    @params:
    - labels: answer labels
    - prediction: predicted labels
    """
    # Calculate confusion matrix
    cm = np.zeros((len(np.unique(labels)), len(np.unique(prediction))))
    for true_label, pred_label in zip(labels, prediction):
        cm[true_label, pred_label] += 1
    
    # Calculate accuracy without LAPJV
    def linear_assignment(cost_matrix):
        # Hungarian algorithm for solving the linear assignment problem
        n = cost_matrix.shape[0]
        row_indices = list(range(n))
        col_indices = list(range(n))
        matches = []
        while row_indices:
            row = row_indices.pop(0)
            min_val = float('inf')
            min_col = None
            for col in col_indices:
                if cost_matrix[row, col] < min_val:
                    min_val = cost_matrix[row, col]
                    min_col = col
            matches.append((row, min_col))
            col_indices.remove(min_col)
        return matches

    def _make_cost_m(cm):
        s = np.max(cm)
        return (- cm + s)

    cost_matrix = _make_cost_m(cm)
    indexes = linear_assignment(cost_matrix)
    js = [e[1] for e in sorted(indexes, key=lambda x: x[0])]
    cm2 = cm[:, js]

    # Calculate accuracy
    accuracy = np.trace(cm2) / np.sum(cm2)
    return accuracy

In [4]:
def get_MCM_score(labels, predictions):
    """
    get precision, recall, f1_score
    @params:
    - labels: answer labels
    - prediction: predicted labels
    """
    accuracy = get_accuracy(labels, predictions)
    precision, recall, f_score, support = precision_recall_fscore_support(labels, predictions, average="macro")

    return {
        "ACC": format(accuracy * 100, ".3f"),
        "PRECISION": format(precision * 100, ".3f"),
        "RECALL": format(recall * 100, ".3f"),
        "F1_SCORE": format(f_score * 100, ".3f")
    }

In [5]:
class VulImageDataset(Dataset):
    """
    Image format code dataset
    """
    def __init__(self, texts, targets, max_len, hidden_size):
        self.texts = texts  # text in image vector format
        self.targets = targets  # target
        self.max_len = max_len  # max number of lines for each code file
        self.hidden_size = hidden_size  # dimension of each code line embedding

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        feature = self.texts[index]
        target = self.targets[index]
        vectors = np.zeros(shape=(3, self.max_len, self.hidden_size))  # image vector (auto-padded with zeros if shorter than max_len)
        for j in range(3):  # 3 channels (degree, katz, closeness) centrality
            for i in range(min(len(feature[0]), self.max_len)):
                vectors[j][i] = feature[j][i]

        return {
            "vector": vectors,
            "targets": torch.tensor(target, dtype=torch.long)
        }

In [6]:
class TextCNN(nn.Module):
    """
    Implementation of VulCNN
    """
    def __init__(self, hidden_size):
        super(TextCNN, self).__init__()
        self.filter_sizes = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)           
        self.num_filters = 32                                        
        classifier_dropout = 0.1
        self.convs = nn.ModuleList(
            [nn.Conv2d(3, self.num_filters, (k, hidden_size)) for k in self.filter_sizes]
        )
        self.dropout = nn.Dropout(classifier_dropout)
        num_classes = 2
        self.fc = nn.Linear(self.num_filters * len(self.filter_sizes), num_classes)  # output layer

    def conv_and_pool(self, x, conv):
        x = F.relu(conv(x)).squeeze(3)
        x = F.max_pool1d(x, x.size(2)).squeeze(2)
        return x

    def forward(self, x):
        out = x.float()
        hidden_state = torch.cat([self.conv_and_pool(out, conv) for conv in self.convs], 1)
        out = self.dropout(hidden_state)
        out = self.fc(out)
        return out, hidden_state

In [7]:
class CNN_Classifier():
    """
    Code Vulnerability Classifier
    """
    def __init__(self, max_len=100, n_classes=2, epochs=100, batch_size = 32, learning_rate = 0.001, \
                    result_save_path = "./data/results", hidden_size = 128):
        self.model = TextCNN(hidden_size)  # VulCNN model
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.max_len = max_len
        self.epochs = epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.model.to(self.device)
        self.hidden_size = hidden_size
        result_save_path = result_save_path + "/" if result_save_path[-1]!="/" else result_save_path
        if not os.path.exists(result_save_path): os.makedirs(result_save_path)
        self.result_save_path = result_save_path + "_epo" + str(epochs) + "_bat" + str(batch_size) + ".result"

    def preparation(self, X_train,  y_train, X_valid, y_valid):
        # create datasets
        self.train_set = VulImageDataset(X_train, y_train, self.max_len, self.hidden_size)
        self.valid_set = VulImageDataset(X_valid, y_valid, self.max_len, self.hidden_size)

        # create data loaders
        self.train_loader = DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True)
        self.valid_loader = DataLoader(self.valid_set, batch_size=self.batch_size, shuffle=True)

        # helpers initialization
        self.optimizer = AdamW(self.model.parameters(), lr=self.learning_rate, correct_bias=False)
        self.scheduler = get_linear_schedule_with_warmup(
            self.optimizer,
            num_warmup_steps=0,
            num_training_steps=len(self.train_loader) * self.epochs
        )
        self.loss_fn = torch.nn.CrossEntropyLoss().to(self.device)  # classification loss function

    def fit(self):
        self.model = self.model.train()
        losses = []
        labels = []
        predictions = []
        scaler = torch.cuda.amp.GradScaler()
        progress_bar = tqdm(self.train_loader, total=len(self.train_loader))
        for data in progress_bar:
            self.optimizer.zero_grad()
            vectors = data["vector"].to(self.device)
            targets = data["targets"].to(self.device)
            with autocast():
                outputs,_  = self.model( vectors )
                loss = self.loss_fn(outputs, targets)
            scaler.scale(loss).backward()
            scaler.step(self.optimizer)
            scaler.update()
            preds = torch.argmax(outputs, dim=1).flatten()           
            
            losses.append(loss.item())
            predictions += list(np.array(preds.cpu()))   
            labels += list(np.array(targets.cpu()))      

            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)

            self.scheduler.step()
            progress_bar.set_description(
                f'loss: {loss.item():.3f}, acc : {(torch.sum(preds == targets)/len(targets)):.3f}')
        train_loss = np.mean(losses)
        score_dict = get_MCM_score(labels, predictions)
        return train_loss, score_dict

    def eval(self):
        print("start evaluating...")
        self.model = self.model.eval()
        losses = []
        pre = []
        label = []
        correct_predictions = 0
        progress_bar = tqdm(self.valid_loader, total=len(self.valid_loader))

        with torch.no_grad():
            for data in progress_bar:
                vectors = data["vector"].to(self.device)
                targets = data["targets"].to(self.device)
                outputs, _ = self.model(vectors)
                loss = self.loss_fn(outputs, targets)
                preds = torch.argmax(outputs, dim=1).flatten()
                correct_predictions += torch.sum(preds == targets)

                pre += list(np.array(preds.cpu()))
                label += list(np.array(targets.cpu()))
                
                losses.append(loss.item())
                progress_bar.set_description(
                f'loss: {loss.item():.3f}, acc : {(torch.sum(preds == targets)/len(targets)):.3f}')
        val_acc = correct_predictions.double() / len(self.valid_set)
        print("val_acc : ",val_acc)
        score_dict = get_MCM_score(label, pre)
        val_loss = np.mean(losses)
        return val_loss, score_dict

    # train model and evaluate
    def train(self):
        learning_record_dict = {}
        train_table = PrettyTable(['typ', 'epo', 'loss', 'ACC', 'PRECISION', 'RECALL', 'F1_SCORE'])
        test_table = PrettyTable(['typ', 'epo', 'loss', 'ACC', 'PRECISION', 'RECALL', 'F1_SCORE'])
        for epoch in range(self.epochs):
            print(f'Epoch {epoch + 1}/{self.epochs}')
            train_loss, train_score = self.fit()
            train_table.add_row(["tra", str(epoch+1), format(train_loss, '.4f')] + [train_score[j] for j in train_score])
            print(train_table)

            val_loss, val_score = self.eval()
            test_table.add_row(["val", str(epoch+1), format(val_loss, '.4f')] + [val_score[j] for j in val_score])
            print(test_table)
            print("\n")
            learning_record_dict[epoch] = {'train_loss': train_loss, 'val_loss': val_loss, \
                    "train_score": train_score, "val_score": val_score}
            sava_data(self.result_save_path, learning_record_dict)
            print("\n")

In [8]:
# function to get data from pickle file
def get_dataframe(pathname:str):
    pathname = pathname + "/" if pathname[-1] != "/" else pathname
    train_df = load_data(pathname + "train.pkl")
    eval_df = load_data(pathname + "test.pkl")
    return train_df, eval_df

In [9]:
hidden_size = 256
data_path = "./data/code_image_data/"
save_path = "./data/results/"

In [10]:
# load train and eval data
train_df, eval_df = get_dataframe(pathname=data_path)

Begin to load data： ./data/code_image_data/train.pkl
Begin to load data： ./data/code_image_data/test.pkl


In [11]:
# reset index after shuffle
train_df.reset_index(drop=True, inplace=True)
eval_df.reset_index(drop=True, inplace=True)

In [12]:
# load classifier
classifier = CNN_Classifier(epochs=100, hidden_size=hidden_size)

In [13]:
# prepare data
classifier.preparation(
    X_train=train_df["data"],
    y_train=train_df["label"],
    X_valid=eval_df["data"],
    y_valid=eval_df["label"]
)

C:\Users\nanal\anaconda3\Lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:
# train
classifier.train()

Epoch 1/100


loss: 0.394, acc : 0.875: 100%|██████████████████████████████████████████████████████| 939/939 [00:41<00:00, 22.64it/s]


+-----+-----+--------+--------+-----------+--------+----------+
| typ | epo |  loss  |  ACC   | PRECISION | RECALL | F1_SCORE |
+-----+-----+--------+--------+-----------+--------+----------+
| tra |  1  | 0.4538 | 75.526 |   74.021  | 72.029 |  72.690  |
+-----+-----+--------+--------+-----------+--------+----------+
start evaluating...


loss: 0.410, acc : 0.750: 100%|██████████████████████████████████████████████████████| 105/105 [00:02<00:00, 37.65it/s]


val_acc :  tensor(0.8294, device='cuda:0', dtype=torch.float64)
+-----+-----+--------+--------+-----------+--------+----------+
| typ | epo |  loss  |  ACC   | PRECISION | RECALL | F1_SCORE |
+-----+-----+--------+--------+-----------+--------+----------+
| val |  1  | 0.3628 | 82.944 |   81.612  | 82.010 |  81.798  |
+-----+-----+--------+--------+-----------+--------+----------+


Begin to save data： ./data/results/_epo100_bat32.result


Epoch 2/100


...

val_acc :  tensor(0.8585, device='cuda:0', dtype=torch.float64)
+-----+-----+--------+--------+-----------+--------+----------+
| typ | epo |  loss  |  ACC   | PRECISION | RECALL | F1_SCORE |
+-----+-----+--------+--------+-----------+--------+----------+
| val |  1  | 0.3628 | 82.944 |   81.612  | 82.010 |  81.798  |
| val |  2  | 0.3190 | 84.382 |   84.240  | 81.679 |  82.632  |
| val |  3  | 0.2911 | 84.113 |   82.799  | 83.798 |  83.209  |
| val |  4  | 0.2827 | 84.143 |   83.962  | 81.421 |  82.366  |
| val |  5  | 0.2800 | 84.502 |   84.905  | 81.351 |  82.561  |
| val |  6  | 0.2595 | 85.372 |   84.540  | 83.764 |  84.116  |
| val |  7  | 0.2480 | 85.881 |   85.394  | 83.897 |  84.527  |
| val |  8  | 0.2566 | 85.462 |   85.962  | 82.449 |  83.670  |
| val |  9  | 0.2402 | 85.671 |   84.551  | 84.779 |  84.662  |
| val |  10 | 0.2316 | 85.971 |   84.888  | 85.051 |  84.968  |
| val |  11 | 0.2267 | 86.841 |   85.632  | 86.889 |  86.130  |
| val |  12 | 0.2255 | 86.841 |   85.735

loss: 0.193, acc : 0.750: 100%|██████████████████████████████████████████████████████| 939/939 [00:41<00:00, 22.67it/s]


+-----+-----+--------+--------+-----------+--------+----------+
| typ | epo |  loss  |  ACC   | PRECISION | RECALL | F1_SCORE |
+-----+-----+--------+--------+-----------+--------+----------+
| tra |  1  | 0.4538 | 75.526 |   74.021  | 72.029 |  72.690  |
| tra |  2  | 0.3448 | 82.347 |   81.104  | 80.823 |  80.957  |
| tra |  3  | 0.3090 | 83.720 |   82.531  | 82.467 |  82.499  |
| tra |  4  | 0.2848 | 84.975 |   83.873  | 83.841 |  83.857  |
| tra |  5  | 0.2706 | 85.661 |   84.562  | 84.711 |  84.635  |
| tra |  6  | 0.2593 | 86.035 |   84.940  | 85.170 |  85.052  |
| tra |  7  | 0.2515 | 86.384 |   85.302  | 85.580 |  85.436  |
| tra |  8  | 0.2426 | 86.554 |   85.461  | 85.831 |  85.636  |
| tra |  9  | 0.2376 | 86.854 |   85.780  | 86.151 |  85.956  |
| tra |  10 | 0.2318 | 87.207 |   86.146  | 86.559 |  86.341  |
| tra |  11 | 0.2266 | 87.590 |   86.548  | 86.982 |  86.753  |
| tra |  12 | 0.2230 | 87.583 |   86.541  | 86.975 |  86.746  |
| tra |  13 | 0.2196 | 87.776 |   86.735

loss: 0.189, acc : 0.875: 100%|██████████████████████████████████████████████████████| 105/105 [00:02<00:00, 36.43it/s]

val_acc :  tensor(0.8582, device='cuda:0', dtype=torch.float64)
+-----+-----+--------+--------+-----------+--------+----------+
| typ | epo |  loss  |  ACC   | PRECISION | RECALL | F1_SCORE |
+-----+-----+--------+--------+-----------+--------+----------+
| val |  1  | 0.3628 | 82.944 |   81.612  | 82.010 |  81.798  |
| val |  2  | 0.3190 | 84.382 |   84.240  | 81.679 |  82.632  |
| val |  3  | 0.2911 | 84.113 |   82.799  | 83.798 |  83.209  |
| val |  4  | 0.2827 | 84.143 |   83.962  | 81.421 |  82.366  |
| val |  5  | 0.2800 | 84.502 |   84.905  | 81.351 |  82.561  |
| val |  6  | 0.2595 | 85.372 |   84.540  | 83.764 |  84.116  |
| val |  7  | 0.2480 | 85.881 |   85.394  | 83.897 |  84.527  |
| val |  8  | 0.2566 | 85.462 |   85.962  | 82.449 |  83.670  |
| val |  9  | 0.2402 | 85.671 |   84.551  | 84.779 |  84.662  |
| val |  10 | 0.2316 | 85.971 |   84.888  | 85.051 |  84.968  |
| val |  11 | 0.2267 | 86.841 |   85.632  | 86.889 |  86.130  |
| val |  12 | 0.2255 | 86.841 |   85.735